# Import necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

2024-04-24 18:08:21.526701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 18:08:21.526794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 18:08:21.696598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# #Read dataset validation , train ,test

In [2]:
train_data = '/kaggle/input/urecamain/Train'
validation_data = '/kaggle/input/urecamain/Vali'
test_data = '/kaggle/input/urecamain/Test'

# Generate data

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 6003 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# Load pretrained Models

In [4]:
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Add classification layers on top of models 

In [5]:
model_efficientnet = Sequential([
    efficientnet_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification for fire detection
])

model_vgg16 = Sequential([
    vgg16_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
      Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1, activation='sigmoid')
])

model_resnet50 = Sequential([
    resnet50_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1, activation='sigmoid')
])

# setup optimizers for all 3 pretrained models

In [6]:
optimizer_efficientnet = Adam(learning_rate=0.01)
optimizer_vgg16 = Adam(learning_rate=0.01)
optimizer_resnet50 = Adam(learning_rate=0.01)

# Compile and train models

In [10]:
loss_function = 'binary_crossentropy'
metrics = ['accuracy']
model_efficientnet.compile(optimizer=optimizer_efficientnet, loss=loss_function, metrics=metrics)
model_vgg16.compile(optimizer=optimizer_vgg16, loss=loss_function, metrics=metrics)
model_resnet50.compile(optimizer=optimizer_resnet50, loss=loss_function, metrics=metrics)
epochs = 25
model_efficientnet.fit(train_generator, epochs=epochs, validation_data=validation_generator)
model_vgg16.fit(train_generator, epochs=epochs, validation_data=validation_generator)
model_resnet50.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/25
  1/188 ━━━━━━━━━━━━━━━━━━━━ 3:47:52 73s/step - accuracy: 0.9375 - loss: 0.1689

W0000 00:00:1713983531.551447     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


105/188 ━━━━━━━━━━━━━━━━━━━━ 1:03 764ms/step - accuracy: 0.9277 - loss: 0.1927

W0000 00:00:1713983611.087380     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 572ms/step - accuracy: 0.9331 - loss: 0.1797

...

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.5050 - loss: 0.6932

W0000 00:00:1713987526.715533     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


188/188 ━━━━━━━━━━━━━━━━━━━━ 149s 525ms/step - accuracy: 0.5050 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 2/25


W0000 00:00:1713987533.822628     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


188/188 ━━━━━━━━━━━━━━━━━━━━ 76s 387ms/step - accuracy: 0.4995 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 74s 381ms/step - accuracy: 0.4874 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 83s 386ms/step - accuracy: 0.4950 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 75s 385ms/step - accuracy: 0.5030 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 6/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 75s 382ms/step - accuracy: 0.4930 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 77s 395ms/step - accuracy: 0.5015 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 76s 390ms/step - accuracy: 0.4932 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 9/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 75s 385ms/step - accuracy: 0.5039 - loss: 0.6932 - val

# Print best Model

In [11]:
eval_results = {}
for model_name, model in zip(['EfficientNetB0', 'VGG16', 'ResNet50'], [model_efficientnet, model_vgg16, model_resnet50]):
    eval_results[model_name] = model.evaluate(test_generator)[1]  # Store only the accuracy
best_model = max(eval_results, key=eval_results.get)
best_accuracy = eval_results[best_model]
print("Best Model:", best_model)
print("Best Accuracy:", best_accuracy)

63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.9586 - loss: 0.1414
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 116ms/step - accuracy: 0.4992 - loss: 0.6932
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.4806 - loss: 0.6933
Best Model: EfficientNetB0
Best Accuracy: 0.9580000042915344
